In [ ]:
import dlib
import cv2
import os
import re
import json
from pylab import *
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
train_frame_folder = './train_sample_videos/'
with open(os.path.join(train_frame_folder, 'metadata.json'), 'r') as file:
    data = json.load(file)
list_of_train_data = [f for f in os.listdir(train_frame_folder) if f.endswith('.mp4')]
detector = dlib.get_frontal_face_detector()
for vid in list_of_train_data:
    count = 0
    cap = cv2.VideoCapture(os.path.join(train_frame_folder, vid))
    frameRate = cap.get(5)
    while cap.isOpened():
        frameId = cap.get(1)
        ret, frame = cap.read()
        if ret != True:
            break
        if frameId % ((int(frameRate)+1)*1) == 0:
            face_rects, scores, idx = detector.run(frame, 0)
            for i, d in enumerate(face_rects):
                x1 = d.left()
                y1 = d.top()
                x2 = d.right()
                y2 = d.bottom()
                crop_img = frame[y1:y2, x1:x2]
                if data[vid]['label'] == 'REAL':
                    cv2.imwrite('./dataset/real/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                elif data[vid]['label'] == 'FAKE':
                    cv2.imwrite('./dataset/fake/'+vid.split('.')[0]+'_'+str(count)+'.png', cv2.resize(crop_img, (128, 128)))
                count+=1


In [2]:
import os
import cv2
import json
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [ ]:
input_shape = (128, 128, 3)
data_dir = './dataset'

real_data = [f for f in os.listdir(data_dir+'/real') if f.endswith('.png')]
fake_data = [f for f in os.listdir(data_dir+'/fake') if f.endswith('.png')]

X = []
Y = []

for img in real_data:
    X.append(img_to_array(load_img(data_dir+'/real/'+img)).flatten() / 255.0)
    Y.append(1)
for img in fake_data:
    X.append(img_to_array(load_img(data_dir+'/fake/'+img)).flatten() / 255.0)
    Y.append(0)

Y_val_org = Y

#Normalization
X = np.array(X)
Y = to_categorical(Y, 2)

#Reshape
X = X.reshape(-1, 128, 128, 3)

#Train-Test split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [3]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model 
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

input_shape = (128, 128, 3)
googleNet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
googleNet_model.trainable = True
model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 2, 2, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 3074      
                                                                 
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


c:\Users\vntsang\.conda\envs\embryo\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [4]:
model.load_weights('./weight/transfer_inception_resnet.h5')

In [7]:
import time
import dlib

class_names = ["Fake", "Real"]
input_shape = (128, 128, 3)
pr_data = []
detector = dlib.get_frontal_face_detector()
cap = cv2.VideoCapture('./dfdc_train_part_46/aagsmskgyw.mp4')
frame_rate = 60
prev = 0
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH);
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT); 
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'avc1')
out = cv2.VideoWriter("output_video.mp4", fourcc, frame_rate, (int(w),int(h)))

num_frame = 0
while cap.isOpened():
    time_elapsed = time.time() - prev
    ret, frame = cap.read()
    if ret != True:
        break
    # print(time_elapsed)
    if time_elapsed > 1./frame_rate:
        face_rects, scores, idx = detector.run(frame, 0)
        for i, d in enumerate(face_rects):
            x1 = d.left()
            y1 = d.top()
            x2 = d.right()
            y2 = d.bottom()
        
        crop_img = frame[y1:y2, x1:x2]
        data = img_to_array(cv2.resize(crop_img, (128, 128))).flatten() / 255.0
        data = data.reshape(-1, 128, 128, 3)
        predict_x=model.predict(data) 
        classes_x=np.argmax(predict_x,axis=1)
        predicted_class = class_names[np.argmax(predict_x)]
        cv2.putText(frame, predicted_class, (150, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 2)
        print(frame.shape)
        # cv2.imwrite('./videos/frame_' + str(num_frame) + '.jpg', frame)
        # num_frame += 1
        out.write(frame)
        prev = time.time()
cap.release()
out.release()
cv2.destroyAllWindows()
            
            

1/1 [==============================] - 0s 470ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 204ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 451ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 210ms/step
(1080, 1920, 3)
1/1 [==============================] - 1s 577ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 202ms/step
(1080, 1920, 3)
1/1 [==============================] - 1s 566ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 205ms/step
(1080, 1920, 3)
1/1 [==============================] - 1s 565ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 229ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 204ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 499ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 401ms/step
(1080, 1920, 3)
1/1 [==============================] - 0s 308ms/step
(1080, 1920, 3)
1/1 [=============================